In [1]:
import torch
from pathlib import Path
from llama_index import download_loader
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index import ServiceContext
from llama_index import StorageContext, load_index_from_storage
from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display


import chromadb
from llama_index.vector_stores import ChromaVectorStore


In [14]:
if torch.cuda.is_available():
    device = torch.cuda.get_device_name(0)
    print("CUDA device:", device)
else:
    print("CUDA is not available")


CUDA device: NVIDIA GeForce RTX 3090


In [2]:
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

In [2]:
llm = LlamaCPP(
    model_path="./mistral-7b-instruct-v0.2.Q5_K_S.gguf",
    temperature=0.4,
    max_new_tokens=512,
    context_window=2048,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


: 

In [4]:
# llm = HuggingFaceLLM(
#     context_window=2048,
#     max_new_tokens=256,
#     system_prompt=system_prompt,
#     generate_kwargs={},
#     query_wrapper_prompt=query_wrapper_prompt,
#     tokenizer_name="microsoft/phi-2",
#     model_name="microsoft/phi-2",
#     device_map="auto",
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     tokenizer_kwargs={"max_length": 2048},
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path('./input.pdf'))

In [3]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model,
    chunk_overlap=128,
)

In [6]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [7]:
query_engine = index.as_query_engine()

In [12]:
response = query_engine.query("What's the point of making myself less accessible?")

Llama.generate: prefix-match hit


In [13]:
display(Markdown(f"<b>{response}</b>"))

<b> Based on the provided context from "The 48 Laws of Power," there are several laws that suggest making oneself less accessible can be a strategic move. Here are some possible reasons:

1. The Mirror Effect (Law 2): By being less accessible, you may reflect the innermost feelings of those around you who crave your attention and time. This can make them feel more connected to you and increase their loyalty.
2. Envy (Law 45 and Law 46): Making yourself less accessible can create an air of mystery and exclusivity, which can fuel envy in others. By revealing weaknesses or imperfections, you may deflect envy and maintain a more positive relationship with those around you.
3. Playing to People's Fantasies (Law 32): Being less accessible can add an element of allure and intrigue, making people want to know more about you and potentially fueling their fantasies.
4. Discover Each Man's Thumbscrew (Law 33): Understanding someone's weaknesses or insecurities can give you leverage in a relationship or interaction. By being less accessible, you may be able to use this knowledge to your advantage while maintaining a sense of mystery.
5. Be Royal in Your Own Fashion (Law 34): Exuding confidence and an air of exclusivity can help you command respect and be treated like a king or queen. Being less accessible can contribute to this image.</b>

In [15]:
index.storage_context.persist()

In [ ]:
#ChromaDB

In [4]:
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("laws_of_power")

In [6]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents,
                                        storage_context=storage_context,
                                        service_context=service_context)

In [7]:
query_engine = index.as_query_engine()
response = query_engine.query("Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


<b> According to "The 48 Laws of Power" book, there isn't a specific law or story dedicated to Queen Elizabeth I in the text provided. However, there is a reference to her in Law 43, "Work on the Hearts and Minds of Others," under the subsection "Transgression of the Law." The example given in the context of this law describes how Emperor Sung of China was able to turn his enemies into loyal friends, and another example mentions Michael III of the Byzantine Empire who placed too much trust in his friend Basilius. In contrast, Queen Elizabeth I is suggested as a successful monarch who won the hearts and minds of others by being mindful of their emotions and responding appropriately, as mentioned earlier. The text does not go into detail about specific events or actions she took to accomplish this, but it implies that her ability to connect with people and respond to their needs effectively was a key factor in her success.</b>

In [5]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("laws_of_power")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model,
                                               llm=llm,
                                               chunk_size=1024,
                                               chunk_overlap=128)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

In [6]:
query_engine = index.as_query_engine()

response = query_engine.query("What's the point of making myself less accessible?")

display(Markdown(f"{response}"))

 Based on the provided context from "The 48 Laws of Power," there are several reasons suggested for making oneself less accessible. According to Law 32, "Play to People's Fantasies," people have a need for fantasies and the more vague and exotic they are, the more captivating. By keeping a certain distance and being less accessible, you can maintain the mystery and allure that comes with those fantasies. Additionally, as per Law 33, "Discover Each Man's Thumbscrew," every person has a weakness or insecurity, and by being less accessible, you may make it more difficult for others to discover your own weaknesses or vulnerabilities. Lastly, according to Law 47, "Do Not Go Past the Mark You Aimed For; In Victory, Learn When To Stop," being less accessible can also help in controlling the pace and outcome of interactions, allowing you to maintain power and influence.

In [4]:
db2 = chromadb.PersistentClient(path="./chroma_db")

chroma_collection = db2.get_or_create_collection("laws_of_power")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    service_context=service_context,
)

In [8]:
query_engine = index.as_query_engine()

response = query_engine.query("Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?")

display(Markdown(f"{response}"))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 The 48 Laws of Power book mentions Queen Elizabeth I in relation to Law 43, "Work on the Hearts and Minds of Others." While the book doesn't tell a comprehensive story about her life, it highlights how Elizabeth I used her charisma and emotional intelligence to maintain power and win over her subjects.

Elizabeth I ascended the throne in 1558 after a period of political instability during which England saw multiple rulers. Her ability to connect with her people was crucial for maintaining stability and securing her position. In contrast, Marie-Antoinette, Queen Elizabeth I's successor, faced the opposite fate due to her lavish lifestyle and disconnection from her people.

The book uses the example of Emperor Sung of China in 959 A.D., who turned his enemies into loyal friends by persuading them to retire and granting them noble titles. Queen Elizabeth I employed a similar tactic, using patronage and favoritism to win over influential figures and maintain their loyalty. She also made sure to present herself as a benevolent ruler, distributing gifts and pardons to her subjects.

In the context of Law 43, the book emphasizes the importance of understanding the emotions and desires of those around you in order to manipulate them to your advantage. Elizabeth I's ability to connect with her people on an emotional level allowed her to maintain power and secure her position as a beloved and effective monarch.